# Box 1: ACC, high EKE

In [1]:
import xarray as xr
import numpy as np
import gsw
import matplotlib.pyplot as plt
import matplotlib as mpl
from importlib import reload
from cmocean import cm as cmo
import xrft
import pandas as pd


import density_funcs as df
import EV_funcs as ef
import filt_funcs as ff
import plot_funcs as pf
import MLD_funcs as mf

In [2]:
reload(df)
reload(ef)
reload(ff)
reload(pf)
reload(mf)

<module 'MLD_funcs' from '/home.ufs/amf2288/argo-intern/MLD_funcs.py'>

## Select and Plot Box

In [ ]:
box_z = xr.open_dataset('202206_boxes/.nc')

In [ ]:
pf.plot_box([[]])

In [ ]:
box_z = mf.get_MLD(box_z)
box_z = mf.add_times(box_z)

In [ ]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
plt.figure(figsize=(11,5))

plt.subplot(1,2,1)
plt.scatter(box_z.month,box_z.year)
plt.yticks(range(2008,2023),range(2008,2023))
plt.xticks(range(1,13),months)
plt.grid()
plt.title('Whole Months & Years')

plt.subplot(1,2,2)
plt.scatter(box_z.month_frac,box_z.year_frac)
plt.yticks(range(2008,2023),range(2008,2023))
plt.xticks(range(1,13),months)
plt.grid()
plt.title('Fractional Months & Years')

## Plot Tracers and Profiles

In [ ]:
n=0
mean_prof = box_z.SPICE.isel(PRES_INTERPOLATED=n).mean(skipna=True)
this_prof = box_z.SPICE.isel(PRES_INTERPOLATED=n) - mean_prof

for n in range(1,len(box_z.PRES_INTERPOLATED)):
    mean_prof      = box_z.SPICE.isel(PRES_INTERPOLATED=n).mean(skipna=True)
    this_prof_next = box_z.SPICE.isel(PRES_INTERPOLATED=n) - mean_prof

    this_prof = xr.concat([this_prof, this_prof_next], dim='PRES_INTERPOLATED')
    
SPICE_anom = this_prof

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(2,3,1)
box_z.CT.plot(x='N_PROF_NEW',y='PRES_INTERPOLATED',cmap=cmo.thermal)
plt.gca().invert_yaxis()
plt.title('Temperature')

plt.subplot(2,3,2)
box_z.SA.plot(x='N_PROF_NEW',y='PRES_INTERPOLATED',cmap=cmo.haline)
plt.gca().invert_yaxis()
plt.title('Salinity')

plt.subplot(2,3,4)
box_z.SIG0.plot(x='N_PROF_NEW',y='PRES_INTERPOLATED',cmap=cmo.dense)
plt.gca().invert_yaxis()
plt.title('Density')

plt.subplot(2,3,5)
box_z.SPICE.plot(x='N_PROF_NEW',y='PRES_INTERPOLATED',cmap=cmo.deep)
plt.gca().invert_yaxis()
plt.title('Spice')

plt.subplot(2,3,6)
SPICE_anom.plot(x='N_PROF_NEW',y='PRES_INTERPOLATED',cmap=cmo.balance)
plt.gca().invert_yaxis()
plt.title('Spice Anom')

plt.subplots_adjust(hspace=0.3,wspace=0.4)

In [ ]:
l1 =  50
l2 = 100
l3 = 200

In [ ]:
CT_l1 = ff.get_filt_single(box_z,l1,'CT')
CT_l2 = ff.get_filt_single(box_z,l2,'CT')
CT_l3 = ff.get_filt_single(box_z,l3,'CT')

SA_l1 = ff.get_filt_single(box_z,l1,'SA')
SA_l2 = ff.get_filt_single(box_z,l2,'SA')
SA_l3 = ff.get_filt_single(box_z,l3,'SA')

SIG0_l1 = ff.get_filt_single(box_z,l1,'SIG0')
SIG0_l2 = ff.get_filt_single(box_z,l2,'SIG0')
SIG0_l3 = ff.get_filt_single(box_z,l3,'SIG0')

SPICE_l1 = ff.get_filt_single(box_z,l1,'SPICE')
SPICE_l2 = ff.get_filt_single(box_z,l2,'SPICE')
SPICE_l3 = ff.get_filt_single(box_z,l3,'SPICE')

In [ ]:
n=20
plt.figure(figsize=(6,10))

plt.subplot(2,2,1)
box_z.CT.isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
CT_l1.where(CT_l1.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
CT_l2.where(CT_l2.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
CT_l3.where(CT_l3.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
plt.hlines(box_z.MLD.isel(N_PROF=n),0,2,color='k',linestyle='--',label='MLD')
plt.gca().invert_yaxis()
plt.title('CT Profile, Filtered')

plt.subplot(2,2,2)
box_z.SA.isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
SA_l1.where(CT_l1.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
SA_l2.where(CT_l2.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
SA_l3.where(CT_l3.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
plt.hlines(box_z.MLD.isel(N_PROF=n),34,35,color='k',linestyle='--',label='MLD')
plt.gca().invert_yaxis()
plt.title('SA Profile, Filtered')

plt.subplot(2,2,3)
box_z.SIG0.isel(N_PROF=n).plot(y='PRES_INTERPOLATED',label='profile')
SIG0_l1.where(CT_l1.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED',label='l=100')
SIG0_l2.where(CT_l2.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED',label='l=200')
SIG0_l3.where(CT_l3.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED',label='l=400')
plt.hlines(box_z.MLD.isel(N_PROF=n),27.25,28,color='k',linestyle='--',label='MLD')
plt.gca().invert_yaxis()
plt.legend()
plt.title('SIG0 Profile, Filtered')

plt.subplot(2,2,4)
box_z.SPICE.isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
SPICE_l1.where(CT_l1.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
SPICE_l2.where(CT_l2.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
SPICE_l3.where(CT_l3.mask==1).isel(N_PROF=n).plot(y='PRES_INTERPOLATED')
plt.hlines(box_z.MLD.isel(N_PROF=n),-0.75,0,color='k',linestyle='--',label='MLD')
plt.gca().invert_yaxis()
plt.title('SPICE Profile, Filtered')

plt.subplots_adjust(wspace=0.5)

## Plot MLD

In [ ]:
SIG0_01 = box_z.SIG0.where(box_z.month<2)                       .dropna('N_PROF')
SIG0_02 = box_z.SIG0.where(box_z.month<3) .where(box_z.month>1) .dropna('N_PROF')
SIG0_03 = box_z.SIG0.where(box_z.month<4) .where(box_z.month>2) .dropna('N_PROF')
SIG0_04 = box_z.SIG0.where(box_z.month<5) .where(box_z.month>3) .dropna('N_PROF')
SIG0_05 = box_z.SIG0.where(box_z.month<6) .where(box_z.month>4) .dropna('N_PROF')
SIG0_06 = box_z.SIG0.where(box_z.month<7) .where(box_z.month>5) .dropna('N_PROF')
SIG0_07 = box_z.SIG0.where(box_z.month<8) .where(box_z.month>6) .dropna('N_PROF')
SIG0_08 = box_z.SIG0.where(box_z.month<9) .where(box_z.month>7) .dropna('N_PROF')
SIG0_09 = box_z.SIG0.where(box_z.month<10).where(box_z.month>8) .dropna('N_PROF')
SIG0_10 = box_z.SIG0.where(box_z.month<11).where(box_z.month>9) .dropna('N_PROF')
SIG0_11 = box_z.SIG0.where(box_z.month<12).where(box_z.month>10).dropna('N_PROF')
SIG0_12 = box_z.SIG0.where(box_z.month<13).where(box_z.month>11).dropna('N_PROF')

SIG0_s = [SIG0_01,SIG0_02,SIG0_03,SIG0_04,SIG0_05,SIG0_06,SIG0_07,SIG0_08,SIG0_09,SIG0_10,SIG0_11,SIG0_12]

In [ ]:
SPICE_01 = box_z.SPICE.where(box_z.month<2)                       .dropna('N_PROF')
SPICE_02 = box_z.SPICE.where(box_z.month<3) .where(box_z.month>1) .dropna('N_PROF')
SPICE_03 = box_z.SPICE.where(box_z.month<4) .where(box_z.month>2) .dropna('N_PROF')
SPICE_04 = box_z.SPICE.where(box_z.month<5) .where(box_z.month>3) .dropna('N_PROF')
SPICE_05 = box_z.SPICE.where(box_z.month<6) .where(box_z.month>4) .dropna('N_PROF')
SPICE_06 = box_z.SPICE.where(box_z.month<7) .where(box_z.month>5) .dropna('N_PROF')
SPICE_07 = box_z.SPICE.where(box_z.month<8) .where(box_z.month>6) .dropna('N_PROF')
SPICE_08 = box_z.SPICE.where(box_z.month<9) .where(box_z.month>7) .dropna('N_PROF')
SPICE_09 = box_z.SPICE.where(box_z.month<10).where(box_z.month>8) .dropna('N_PROF')
SPICE_10 = box_z.SPICE.where(box_z.month<11).where(box_z.month>9) .dropna('N_PROF')
SPICE_11 = box_z.SPICE.where(box_z.month<12).where(box_z.month>10).dropna('N_PROF')
SPICE_12 = box_z.SPICE.where(box_z.month<13).where(box_z.month>11).dropna('N_PROF')

SPICE_s = [SPICE_01,SPICE_02,SPICE_03,SPICE_04,SPICE_05,SPICE_06,SPICE_07,SPICE_08,SPICE_09,SPICE_10,SPICE_11,SPICE_12]

In [ ]:
plt.figure(figsize=(8,5))
colors = plt.cm.twilight_shifted(np.linspace(0,1,12))

plt.subplot(1,2,1)
for n in range(0,len(SIG0_s)):
    SIG0_mean = SIG0_s[n].mean('N_PROF')
    SIG0_mean.plot(y='PRES_INTERPOLATED',label='{}'.format(months[n]),color=colors[n])
    plt.plot(SIG0_mean.sel(PRES_INTERPOLATED=SIG0_s[n].MLD.mean('N_PROF'),method='nearest'),
             SIG0_s[n].MLD.mean('N_PROF'),marker='o',color=colors[n])
plt.ylim(0,250)
plt.xlim(26.9,27.6)
plt.yticks(range(0,250,50),range(0,250,50))
plt.grid()
plt.gca().invert_yaxis()
plt.legend()
plt.title('Density Profiles')

plt.subplot(1,2,2)
for n in range(0,len(SIG0_s)):
    SPICE_mean = SPICE_s[n].mean('N_PROF')
    SPICE_mean.plot(y='PRES_INTERPOLATED',label='{}'.format(months[n]),color=colors[n])
    plt.plot(SPICE_mean.sel(PRES_INTERPOLATED=SPICE_s[n].MLD.mean('N_PROF'),method='nearest'),
             SPICE_s[n].MLD.mean('N_PROF'),marker='o',color=colors[n])
plt.ylabel('')
plt.ylim(0,250)
plt.xlim(-0.85,-.4)
plt.yticks(range(0,250,50),range(0,250,50))
plt.grid()
plt.gca().invert_yaxis()
plt.title('Spice Profiles')

### DOES MEDIAN MLD LOOK BETTER THAN MEAN??

### EXCERCISE: TS PLOT WITH MEAN CT/SA AT ML BY SEASON

## Plot Spectra

In [ ]:
SIG0_spec  = xrft.power_spectrum(box_z.SIG0.mean('N_PROF'), dim='PRES_INTERPOLATED',window='hamming')
SPICE_spec = xrft.power_spectrum(box_z.SPICE.mean('N_PROF'),dim='PRES_INTERPOLATED',window='hamming')
SIG0_m  = SIG0_spec.freq_PRES_INTERPOLATED
SPICE_m = SPICE_spec.freq_PRES_INTERPOLATED

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
SIG0_spec.plot(color='k')
plt.plot(SIG0_m, 1e-8*SIG0_m**-2,'--',color='tab:blue',label='m=-2')
plt.plot(SIG0_m, 5e-9*SIG0_m**-3,'--',color='tab:red', label='m=-3')
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.title('Density Spectra')


plt.subplot(1,2,2)
SPICE_spec.plot(color='k')
plt.plot(SPICE_m, 1e-8*SPICE_m**(-2),'--',color='tab:blue',label='m=-2')
plt.plot(SPICE_m, 5e-10*SPICE_m**(-3),'--',color='tab:red', label='m=-3')
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.title('Spice Spectra')

## Plot Ratios

In [ ]:
mke1_d = ef.get_MV(box_z,l1,variable='SIG0')
mke2_d = ef.get_MV(box_z,l2,variable='SIG0')
mke3_d = ef.get_MV(box_z,l3,variable='SIG0')

eke1_d = ef.get_EV(box_z,l1,variable='SIG0')
eke2_d = ef.get_EV(box_z,l2,variable='SIG0')
eke3_d = ef.get_EV(box_z,l3,variable='SIG0')

mke1_s = ef.get_MV(box_z,l1,variable='SPICE')
mke2_s = ef.get_MV(box_z,l2,variable='SPICE')
mke3_s = ef.get_MV(box_z,l3,variable='SPICE')

eke1_s = ef.get_EV(box_z,l1,variable='SPICE')
eke2_s = ef.get_EV(box_z,l2,variable='SPICE')
eke3_s = ef.get_EV(box_z,l3,variable='SPICE')

### CHECK MASK KEYWORD IN GET_MV FUNCTION

In [ ]:
MKE0_d = eke1_d.where(eke1_d.mask==1)
MKE1_d = mke1_d.where(mke1_d.mask==1) - mke2_d.where(mke2_d.mask==1)
MKE2_d = mke2_d.where(mke2_d.mask==1) - mke3_d.where(mke3_d.mask==1)
MKE3_d = mke3_d.where(mke3_d.mask==1)

EKE0_d = eke1_d.where(eke1_d.mask==1)
EKE1_d = eke2_d.where(eke2_d.mask==1) - eke1_d.where(eke1_d.mask==1)
EKE2_d = eke3_d.where(eke3_d.mask==1) - eke2_d.where(eke2_d.mask==1)
EKE3_d = mke3_d.where(mke3_d.mask==1)

MKE0_s = eke1_s.where(eke1_d.mask==1)
MKE1_s = mke1_s.where(mke1_d.mask==1) - mke2_s.where(mke2_d.mask==1)
MKE2_s = mke2_s.where(mke2_d.mask==1) - mke3_s.where(mke3_d.mask==1)
MKE3_s = mke3_s.where(mke3_d.mask==1)

EKE0_s = eke1_s.where(eke1_d.mask==1)
EKE1_s = eke2_s.where(eke2_d.mask==1) - eke1_s.where(eke1_d.mask==1)
EKE2_s = eke3_s.where(eke3_d.mask==1) - eke2_s.where(eke2_d.mask==1)
EKE3_s = mke3_s.where(mke3_d.mask==1)

In [ ]:
plt.figure(figsize=(5,10))

plt.subplot(2,2,1)
MKE0_d.mean('N_PROF').plot(y='PRES_INTERPOLATED')
MKE1_d.mean('N_PROF').plot(y='PRES_INTERPOLATED')
MKE2_d.mean('N_PROF').plot(y='PRES_INTERPOLATED')
MKE3_d.mean('N_PROF').plot(y='PRES_INTERPOLATED')
plt.xscale('log')
plt.yticks(range(0,2001,250),range(0,2001,250))
plt.gca().invert_yaxis()
plt.title('MKEs, Density')

plt.subplot(2,2,2)
EKE0_d.mean('N_PROF').plot(y='PRES_INTERPOLATED')
EKE1_d.mean('N_PROF').plot(y='PRES_INTERPOLATED')
EKE2_d.mean('N_PROF').plot(y='PRES_INTERPOLATED')
EKE3_d.mean('N_PROF').plot(y='PRES_INTERPOLATED')
plt.xscale('log')
plt.yticks(range(0,2001,250),range(0,2001,250))
plt.gca().invert_yaxis()
plt.title('EKEs, Density')

plt.subplot(2,2,3)
MKE0_s.mean('N_PROF').plot(y='PRES_INTERPOLATED')
MKE1_s.mean('N_PROF').plot(y='PRES_INTERPOLATED')
MKE2_s.mean('N_PROF').plot(y='PRES_INTERPOLATED')
MKE3_s.mean('N_PROF').plot(y='PRES_INTERPOLATED')
plt.xscale('log')
plt.yticks(range(0,2001,250),range(0,2001,250))
plt.gca().invert_yaxis()
plt.title('MKEs, Spice')

plt.subplot(2,2,4)
EKE0_s.mean('N_PROF').plot(y='PRES_INTERPOLATED',label='bin 0, <{}m'.format(l1))
EKE1_s.mean('N_PROF').plot(y='PRES_INTERPOLATED',label='bin 1, {}m - {}m'.format(l1,l2))
EKE2_s.mean('N_PROF').plot(y='PRES_INTERPOLATED',label='bin 2, {}m - {}m'.format(l2,l3))
EKE3_s.mean('N_PROF').plot(y='PRES_INTERPOLATED',label='bin 3, >{}m'.format(l3))
plt.xscale('log')
plt.yticks(range(0,2001,250),range(0,2001,250))
plt.gca().invert_yaxis()
plt.legend(bbox_to_anchor=(1,0.3))
plt.title('EKEs, Spice')

plt.subplots_adjust(wspace=0.5)

### ADD LINES FOR SHALLOWEST AND DEEPEST MIXED LAYER

In [ ]:
R_l1_d = (EV_l1_d / MV_l1_d).where(EV_l1_d.mask==1).mean('N_PROF')
R_l2_d = (EV_l2_d / MV_l2_d).where(EV_l2_d.mask==1).mean('N_PROF')
R_l3_d = (EV_l3_d / MV_l3_d).where(EV_l3_d.mask==1).mean('N_PROF')

R_l1_s = (EV_l1_s / MV_l1_s).where(EV_l1_s.mask==1).mean('N_PROF')
R_l2_s = (EV_l2_s / MV_l2_s).where(EV_l2_s.mask==1).mean('N_PROF')
R_l3_s = (EV_l3_s / MV_l3_s).where(EV_l3_s.mask==1).mean('N_PROF')